In [1]:
# Download the Iris CSV data if not present
import pandas as pd
import os

# Data from http://archive.ics.uci.edu/ml/datasets/Abalone
test_data = 'abalone.csv'
names = ['sex', 'length', 'diameter', 'height',
         'whole_weight', 'shucked_weight', 'viscera_weight',
         'shell_weight', 'rings']

if not os.path.exists(test_data):
    data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data', names=names)
    abalone = pd.concat([data]*500, ignore_index=True)
    abalone.to_csv(test_data)

In [2]:
# Create Spark session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Demo") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .getOrCreate()


In [9]:
df = spark.read.csv(test_data, header=True, inferSchema=True)
#df.show()

DataFrame[_c0: int, sex: string, length: double, diameter: double, height: double, whole_weight: double, shucked_weight: double, viscera_weight: double, shell_weight: double, rings: int]

In [4]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

@udf(returnType=FloatType())
def square(s):
    # input arg is a floating point
    return s * s

#df2 = df.withColumn("length_squared", square(df['length']))
#df2.show()

In [10]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import pandas_udf

@pandas_udf(returnType=FloatType())
def pd_square(s):
    # input arg is a Pandas Series object (e.g. a 1d array)
#     raise OSError(s.shape)
    return s * s

#df2 = df.withColumn("length_squared", pd_square(df['length']))
#df2.show()

In [11]:
# Disable caching of results
#square_no_opt = square.asNondeterministic()
#pd_square_no_opt = pd_square.asNondeterministic()

%time df.withColumn("length_squared", square(df['length'])).summary()
%time df.withColumn("length_squared", pd_square(df['length'])).summary()

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 11.5 s
CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 11.9 s


DataFrame[summary: string, _c0: string, sex: string, length: string, diameter: string, height: string, whole_weight: string, shucked_weight: string, viscera_weight: string, shell_weight: string, rings: string, length_squared: string]